In [6]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import librosa
import IPython.display as ipd #음성파일 load 확인

#### Train 데이터 불러오기

##### 일반 test 데이터 불러오기

In [8]:
#음성파일 path 설정하기
train_audio_path = 'tensorflow-speech-recognition-challenge/train/train/audio/'

In [9]:
# tqdm : process bar 만들어주는 library
from tqdm import tqdm_notebook

all_wave = []
all_label = []

#train_audio_path = 'tensorflow-speech-recognition-challenge/train/train/audio/'
label = os.listdir(train_audio_path) # 하위 디렉토리 이름들 가져오기

#전체 파일들 가져오기
for i in tqdm_notebook(label) :
    wave = [a for a in os.listdir(train_audio_path+'/'+i) if a.endswith('.wav')]
    for wav in wave :
        #파일 한개 로드하기
        #sr : sampling rate 설정하기 ( default rate :  22500 )
        sample, sample_rate = librosa.load(train_audio_path+'/'+i+'/'+wav, sr=16000)
        # sr을 바꿔서 resampling
        sample = librosa.resample(sample, sample_rate, 8000)
        if (len (sample)==8000):
            all_wave.append(sample)
            all_label.append(i)

<ipython-input-9-b56c21eb2d11>:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(label) :


##### silence로 사용할 소음 데이터 불러오기

In [20]:
silence_audio_path = 'tensorflow-speech-recognition-challenge/train/train/audio/_background_noise_/'

In [51]:
sil_label = []
sil_wave = []

#전체 파일들 가져오기
wave = [a for a in os.listdir(silence_audio_path) if a.endswith('.wav')]

for wav in tqdm_notebook(wave) :
    #파일 한개 로드하기
    #sr : sampling rate 설정하기 ( default rate :  22500 )
    sample, sample_rate = librosa.load(silence_audio_path+wav, sr=16000)
    # sr을 바꿔서 resampling
    sample = librosa.resample(sample, sample_rate, 8000)
    
    for i in range(8000,len(sample),8000):
        sil_wave.append(sample[i-8000:i])
        sil_label.append('silence')

<ipython-input-51-996d27f6e537>:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for wav in tqdm_notebook(wave) :


#### 데이터 전처리

In [11]:
# 차원 변환
all_wave = np.array(all_wave).reshape(-1, 8000, 1)
sil_wave = np.array(sil_wave).reshape(-1, 8000, 1)

(58252, 8000, 1)

In [75]:
np.array(all_wave).shape

(58252, 8000, 1)

In [76]:
np.array(sil_wave).shape

(396, 8000, 1)

In [79]:
total_wave = np.concatenate((all_wave,sil_wave))

In [80]:
np.array(total_wave).shape

(58648, 8000, 1)

In [81]:
total_label = np.concatenate((all_label,sil_label))

In [84]:
from sklearn.preprocessing import LabelEncoder

#문자로 존재하는 label을 숫자화
le = LabelEncoder()
y = le.fit_transform(total_label)

In [85]:
le.classes_

array(['bed', 'bird', 'cat', 'dog', 'down', 'eight', 'five', 'four', 'go',
       'happy', 'house', 'left', 'marvin', 'nine', 'no', 'off', 'on',
       'one', 'right', 'seven', 'sheila', 'silence', 'six', 'stop',
       'three', 'tree', 'two', 'up', 'wow', 'yes', 'zero'], dtype='<U7')

In [87]:
from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(total_wave, y, test_size=0.2, random_state=1, stratify=y)

#### 학습

In [14]:
from keras.models import Model
from keras.layers import Conv1D, Input , MaxPooling1D, Dense, Dropout, Flatten

inputs = Input(shape=(8000, 1))

conv = Conv1D(8, 13, activation='relu')(inputs)
maxpooling = MaxPooling1D(3)(conv) #지엽적인 정보 제거
drop_out = Dropout(0.3) (maxpooling)

conv = Conv1D(32, 9, activation='relu')(drop_out)
maxpooling = MaxPooling1D(3)(conv) #지엽적인 정보 제거
drop_out = Dropout(0.3) (maxpooling)

conv = Conv1D(64, 7, activation='relu')(drop_out)
maxpooling = MaxPooling1D(3)(conv) #지엽적인 정보 제거
drop_out = Dropout(0.3) (maxpooling)

conv = Conv1D(64, 7, activation='relu')(drop_out)
maxpooling = MaxPooling1D(3)(conv) #지엽적인 정보 제거
drop_out = Dropout(0.3) (maxpooling)

flatten = Flatten()(drop_out) #차원축소

dense = Dense(256, activation='relu')(flatten) 
drop_out = Dropout(0.3)(dense)

dense = Dense(128, activation='relu') (drop_out)
drop_out = Dropout(0.3) (dense) #정답클래스의 개수가 많아지면 dese 추가

outputs = Dense(len(label), activation='softmax')(drop_out)

model = Model(inputs, outputs)

##### 음성데이터 학습

In [88]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['acc'])

model.fit(x_train, y_train, epochs=10, batch_size=64, validation_data=(x_valid, y_valid))

Epoch 1/10
734/734 [==============================] - 2120s 3s/step - loss: 1.2776 - acc: 0.6815 - val_loss: 0.6164 - val_acc: 0.8229
Epoch 2/10
734/734 [==============================] - 1907s 3s/step - loss: 0.7158 - acc: 0.7898 - val_loss: 0.5783 - val_acc: 0.8320
Epoch 3/10
734/734 [==============================] - 1950s 3s/step - loss: 0.6637 - acc: 0.8008 - val_loss: 0.5974 - val_acc: 0.8252
Epoch 4/10
734/734 [==============================] - 2049s 3s/step - loss: 0.6486 - acc: 0.8049 - val_loss: 0.5465 - val_acc: 0.8415
Epoch 5/10
734/734 [==============================] - 2054s 3s/step - loss: 0.6053 - acc: 0.8161 - val_loss: 0.5301 - val_acc: 0.8449
Epoch 6/10
734/734 [==============================] - 1990s 3s/step - loss: 0.5944 - acc: 0.8183 - val_loss: 0.4993 - val_acc: 0.8575
Epoch 7/10
734/734 [==============================] - 2065s 3s/step - loss: 0.5905 - acc: 0.8208 - val_loss: 0.6545 - val_acc: 0.8098
Epoch 8/10
734/734 [==============================] - 2054s 3s

#### test 데이터 처리하기

In [59]:
test_audio_path = 'tensorflow-speech-recognition-challenge/test/test/audio/'

In [61]:
from tqdm import tqdm_notebook

test_wave = []

wave = os.listdir(test_audio_path)

#전체 파일들 가져오기
for wav in tqdm_notebook(wave) :
    test, test_rate = librosa.load(test_audio_path+wav, sr=16000)
    sample = librosa.resample(test, test_rate, 8000)
    #1초로 잘라서 추가하기
    if (len (sample)==8000):
        test_wave.append(sample)


<ipython-input-61-8add6cc927c2>:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for wav in tqdm_notebook(wave) :


In [89]:
len(test_wave)

158538

In [97]:
# 테스트 데이터 불러오기 : happy 음성
test_audio_path = 'tensorflow-speech-recognition-challenge/test_11/'
test, test_rate = librosa.load(test_audio_path+'happy.wav', sr=8000)
#test = librosa.resample(test, test_rate, 8000)

In [98]:
# 음성 재생
ipd.Audio(test, rate=8000)

In [99]:
#차원 변환
test = np.array(test).reshape(-1, 8000, 1)

In [100]:
ans_num = model.predict(test).argmax()

In [101]:
le.classes_[ans_num]

'happy'

In [93]:
# 답 리스트에 추가하기
answer = []
answer_list = [4, 8, 11, 14, 15, 16, 18, 21, 23, 27, 29]
    
for wav in tqdm_notebook(test_wave) :
    #예측하고 답 추가하기
    ans_num = model.predict(wav).argmax()
    if ans_num in answer_list :
        answer.append(le.classes_[ans_num]) 
    else:
        answer.append('unknown')

<ipython-input-93-682ee36908c4>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for wav in tqdm_notebook(test_wave) :


ValueError: in user code:

    c:\users\kbm06\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\keras\engine\training.py:1147 predict_function  *
        outputs = self.distribute_strategy.run(
    c:\users\kbm06\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    c:\users\kbm06\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    c:\users\kbm06\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    c:\users\kbm06\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\keras\engine\training.py:1122 predict_step  **
        return self(x, tra